In [7]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [8]:
#Stochastic Petri Net Example 1
# Petri net modeled as queueing network
# Note: This is a queueing network representation of Petri net behavior
model = Network('model')

In [9]:
#Petri net with 3 places and 2 transitions
num_places = 3
num_transitions = 2 #Places represented as queues/delays
places = []
for i in range(num_places):
    if i == 0:
        place = Delay(model, f'Place{i}')  #Source place
    else:
        place = Queue(model, f'Place{i}', SchedStrategy.INF)  #Infinite server
    places.append(place) #Transitions represented as service processes
jobclass = ClosedClass(model, 'Tokens', 4, places[0], 2)

In [10]:
#Transition firing rates (service rates)
for i, place in enumerate(places):
    firing_rate = 1.0 + i * 0.2
    place.setService(jobclass, Exp(firing_rate))

In [11]:
#Petri net structure (simplified as routing)
P = model.initRoutingMatrix() #Cyclic structure representing token flow
for i in range(num_places - 1):
    P.set(jobclass, jobclass, places[i], places[i+1], 1.0)
#Close the loop
P.set(jobclass, jobclass, places[-1], places[0], 1.0)
model.link(P)

In [12]:
# Solve Petri net representation
solver = SolverCTMC(model)
avgTable = solver.getAvgTable()

  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Place0   Tokens  1.5701  1.5701  1.0000  1.0000  1.5701  1.5701
1  Place1   Tokens  1.3084  1.3084  0.8333  0.8333  1.5701  1.5701
2  Place2   Tokens  1.1215  1.1215  0.7143  0.7143  1.5701  1.5701
